In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!ls drive/MyDrive/person_segmentation


model.h5


In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

import PIL.Image
import time

In [4]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def load_img2(img):
  max_dim = 512
  # img = plt.imread(path_to_img)
  img = tf.convert_to_tensor(img, dtype=tf.uint8)
  img = tf.image.convert_image_dtype(img, tf.float32)
  # print(img)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape , tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  print(type(tensor))
  return PIL.Image.fromarray(tensor)

In [144]:

""" Global parameters """
H = 512
W = 512

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """

    """ Loading Model """
    model = tf.keras.models.load_model("drive/MyDrive/person_segmentation/model.h5")
    # model.summary()

    """ Video path """
    video_name = "video-1"
    video_path = "drive/MyDrive/person_segmentation/man_running.mp4"

    """ Reading frames """
    vs = cv2.VideoCapture(video_path)
    _, frame = vs.read()
    h, w, _ = frame.shape
    print(h,w)
    vs.release()

    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    out = cv2.VideoWriter(f'drive/MyDrive/person_segmentation/result_videos.avi', fourcc, 30, (w, h), True)

    cap = cv2.VideoCapture(video_path)


640 1280


In [145]:
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

In [146]:
mask_list = []
frame_list = []
counter = 0
style = plt.imread("style.jpg")
style = load_img2(style)
while True:
        ret, frame = cap.read()
        frame_list.append(frame)

        if ret == False:
            cap.release()
            out.release()
            break
        image = load_img2(frame)
        stylized_image = hub_model(tf.constant(image), tf.constant(style))[0]
        stylized_image = (stylized_image[0]*255).numpy().astype(np.uint8)
        # plt.imshow((stylized_image[0]*255).numpy().astype(np.uint8))
        # out.write((stylized_image[0]*255).numpy().astype(np.uint8))

        h, w, _ = frame.shape
        ori_frame = frame
        frame = cv2.resize(frame, (W, H)) ## (H, W, 3)
        frame = np.expand_dims(frame, axis=0) ## (1, H, W, 3)
        frame = frame / 255.0

        mask = model.predict(frame, verbose=0)[0][:,:,-1]
        mask = cv2.resize(mask, (w, h)) ## (h, w)
        # mask = mask.astype(np.float32)
        mask = np.expand_dims(mask, axis=-1)    ## (h, w, 1)
        mask = mask * 255
        mask = mask // 150
        mask = mask.astype(np.uint8)
        bg = mask.copy()
        # print(mask.shape)
        # print(mask)
        detected = mask ^ 1
        # unique_values, counts = np.unique(bg, return_counts=True)
        # for value, count in zip(unique_values, counts):
        #     print(f"Value {value} appears {count} times.")

        # unique_values, counts = np.unique(detected, return_counts=True)
        # for value, count in zip(unique_values, counts):
        #     print(f"Value {value} appears {count} times.")


        bg = np.concatenate([bg,bg,bg], axis=-1)
        # print(stylized_image.shape)
        # print(bg.shape)
        bg = bg * stylized_image

        detected = np.concatenate([detected, detected, detected], axis=-1)
        # print(detected.shape)
        # print(ori_frame.shape)
        detected = detected * ori_frame

        final_frame = detected+bg
        # print(final_frame.dtype)
        out.write(final_frame)

        # photo_mask = mask
        # background_mask = np.abs(1-mask) ## (h, w, 1)
        # masked_frame = ori_frame * photo_mask

        # background_mask = np.concatenate([background_mask, background_mask, background_mask], axis=-1) ## (h, w, 3)
        # background_mask = background_mask * [255, 0, 255]
        # final_frame = masked_frame + background_mask
        # final_frame = final_frame.astype(np.uint8)
        # print(final_frame.shape)
        # plt.imshow(final_frame[:,:,2]//150, cmap="gray")
        # mask_list.append(final_frame)
        # out.write((stylized_image[0]*255).numpy().astype(np.uint8))
        # print(type(final_frame), final_frame.dtype)
        # print(type(stylized_image[0].numpy()), stylized_image[0].numpy().dtype)
        # plt.imshow(stylized_image[0].numpy())

        # print(final_frame.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27